In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/y_test/

/content/drive/MyDrive/y_test


In [3]:
import pandas as pd

articles_test_df = pd.read_csv("data/articles_data/data_set_4.csv")
articles_test_df.head()

,Unnamed: 0,Link,Text,Label
0,0,https://t.me/c/1376264484/328,"Просто слухайте цей діалог. Ні, це не нарізка ...",True
1,1,https://t.me/c/1376264484/338,️ Рубль став найнестабільнішою валютою у всьом...,True
2,2,https://t.me/c/1376264484/350,Перше звернення мера Мелітополя Івана Федорова...,True
3,3,https://t.me/c/1376264484/361,"Росія загрожує Боснії ""українським сценарієм"" ...",True
4,4,https://t.me/c/1376264484/366,"Енергоатом повідомив, що окупанти пошкодили ви...",True


In [4]:
news_data_test_df = pd.read_csv("data/articles_data/news_data.csv")
news_data_test_df.head()

,Unnamed: 0,Text,Label
0,0,В Україні запустять механізм перевірки наявнос...,True
1,1,У Флориді через рекордно низькі температури на...,True
2,2,ТГ-канал FUCKТИ повідомив про бомбардування Ки...,False
3,3,Окупований Маріуполь не здається. Юнак вийшов...,True
4,4,"Чернігів, Кропивницький і область — повітряна...",True


In [5]:
city_codes_df = pd.read_csv("data/city_codes.csv")
city_codes_df.head()

,Unnamed: 0,Перший рівень,Другий рівень,Третій рівень,Четвертий рівень,Додатковий рівень,Категорія об’єкта,Назва об’єкта
0,3,UA01000000000013043,NaN,NaN,NaN,NaN,O,Автономна Республіка Крим
1,4,UA01000000000013043,UA01020000000022387,NaN,NaN,NaN,P,Бахчисарайський
2,5,UA01000000000013043,UA01020000000022387,UA01020010000048857,NaN,NaN,H,Андріївська
3,6,UA01000000000013043,UA01020000000022387,UA01020010000048857,UA01020010010075540,NaN,C,Андріївка
4,7,UA01000000000013043,UA01020000000022387,UA01020010000048857,UA01020010020030666,NaN,X,Сонячний


In [6]:
city_codes_df.sample(10)["Назва об’єкта"].values

array(['Меліоративне', 'Веселе', 'Новоархангельська', 'Кочичине',
       'Пожарна Балка', 'Біликівці', 'Іванопіль', 'Дружелюбівка',
       'Войнин', 'Воронівка'], dtype=object)

# **ngram search**

In [9]:
import time
from src.data_utils import strings_to_matrix, generate_dummy_data, build_vocabulary, build_vocabulary, compute_cosine_similarity_gpu
import numpy as np

def run_pipeline():
    print("--- 1. Generating Data ---")
    queries, db_cities, noisy_targets, truth_map = generate_dummy_data(num_queries=1000, num_db=30000)
    print(f"Queries: {len(queries)}, Database size: {len(db_cities)}")

    print("\n--- 2. Vectorization (CPU) ---")
    t0 = time.time()

    # Build vocab from everything to ensure all ngrams are covered
    ngram_map, _ = build_vocabulary(queries + db_cities, n=3)
    print(f"Vocabulary Size (N-grams): {len(ngram_map)}")

    # Create matrices
    query_matrix = strings_to_matrix(queries, ngram_map, n=3)
    db_matrix = strings_to_matrix(db_cities, ngram_map, n=3)

    print(f"Vectorization took: {time.time() - t0:.4f}s")

    print("\n--- 3. Similarity Calculation (GPU) ---")
    t1 = time.time()

    # Get distance matrix (Queries x DB)
    dist_matrix = compute_cosine_similarity_gpu(query_matrix, db_matrix)

    calc_time = time.time() - t1
    print(f"GPU Calculation took: {calc_time:.4f}s")

    print("\n--- 4. Results Analysis ---")
    # Find indices of minimum distance for every query
    best_match_indices = np.argmin(dist_matrix, axis=1)

    print("### Example Matches for Noisy Inputs ###")
    for noisy_word in noisy_targets:
        q_idx = queries.index(noisy_word)
        best_db_idx = best_match_indices[q_idx]

        found_name = db_cities[best_db_idx]
        distance = dist_matrix[q_idx, best_db_idx]
        actual_base = truth_map[noisy_word]

        print(f"Input: '{noisy_word}'")
        print(f"  -> Truth:     {actual_base}")
        print(f"  -> Found:     '{found_name}' (Dist: {distance:.4f})")
        print("-" * 30)

if __name__ == "__main__":
    # Check if we are in Colab or have CuPy installed
    if True:
        run_pipeline()
    else:
        print("Please install 'cupy-cuda12x' (or appropriate version) to run this code.")

--- 1. Generating Data ---
Queries: 1000, Database size: 30000

--- 2. Vectorization (CPU) ---
Vocabulary Size (N-grams): 181
Vectorization took: 0.2687s

--- 3. Similarity Calculation (GPU) ---
GPU Calculation took: 0.1114s

--- 4. Results Analysis ---
### Example Matches for Noisy Inputs ###
Input: 'Одrеса'
  -> Truth:     Одеса
  -> Found:     'Одеса' (Dist: 0.4523)
------------------------------
Input: 'Баqxхчисарайський'
  -> Truth:     Бахчисарайський
  -> Found:     'Бахчисарайський' (Dist: 0.1859)
------------------------------
Input: 'Лyьвова'
  -> Truth:     Львова
  -> Found:     'Львова' (Dist: 0.3828)
------------------------------
Input: 'Херсоvqн'
  -> Truth:     Херсон
  -> Found:     'Херсон' (Dist: 0.4226)
------------------------------
Input: 'Київськtий Район'
  -> Truth:     Київський Район
  -> Found:     'Київський Район' (Dist: 0.1609)
------------------------------
